### Import necessary libraries and packages

In [50]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
import webbrowser
from tqdm.notebook import tqdm_notebook, tqdm
from bs4 import BeautifulSoup
import time
import ast

### Get authentication code from redirect_uri

In [4]:

#stackapps 
client_id =  "stackapp_clien_id"

redirect_uri = "https://stackexchange.com/oauth/login_success"
scope = "no_expiry"
auth_url = f"https://stackoverflow.com/oauth?client_id={client_id}&scope={scope}&redirect_uri={redirect_uri}"

import subprocess

auth_url = f"https://stackoverflow.com/oauth?client_id={client_id}&scope={scope}&redirect_uri={redirect_uri}"
#subprocess.run(["open", auth_url])  # On macOS
subprocess.run(["xdg-open", auth_url])  # On Linux
#subprocess.run(["start", auth_url], shell=True)  # On Windows




/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://stackoverflow.com/oauth?client_id=29766&scope=no_expiry&redirect_uri=https://stackexchange.com/oauth/login_success'


CompletedProcess(args=['xdg-open', 'https://stackoverflow.com/oauth?client_id=29766&scope=no_expiry&redirect_uri=https://stackexchange.com/oauth/login_success'], returncode=3)

In [ ]:

client_secret = "stackapp client_secret"
redirect_uri = "https://stackexchange.com/oauth/login_success"

#authentication code from redirect_uri
code = "authentication code"

# Exchange authorization code for access token
token_url = "https://stackoverflow.com/oauth/access_token"
data = {
    'client_id': client_id,
    'client_secret': client_secret,
    'code': code,
    'redirect_uri': redirect_uri,
}

response = requests.post(token_url, data=data)

# Parse the access token from the response
access_token = response.text.split('=')
print("Access Token:", access_token)


### Class to scrap stackexchange data

In [7]:
class StackExchangeScrapper():
    def __init__(self,access_token: object, app_key: object):
        self.access_token = access_token
        self.app_key = app_key
        self.qna_df = ""
    

    def __get_total_question(self, tag, site):
        # Define the base URL for the StackExchange API
        url = "https://api.stackexchange.com/2.3/questions"


        params = {
            "tagged": tag,  # Tag for pandas-related questions
            "site": site,
            "filter": "total",  # Custom filter to include question and accepted answers
            'access_token': self.access_token,
            'key': self.app_key }
                            

        # Make the request
        response = requests.get( url, params=params)
        data = response.json()

        # Extract the total number of questions
        total_questions = data['total']

        # Output the result
        #print(f"Total number of questions tagged with 'python': {total_questions}")
        return total_questions
    
    def __get_question_answer(self, tag, site, page, pagesize):
        # Define the API URL for pandas questions
        url = "https://api.stackexchange.com/2.3/questions"

        # Parameters for the API call
        params = {
            "order": "desc",
            "sort": "votes",
            "tagged": tag,  # Tag for pandas-related questions
            "site": site,
            #"fromdate":"1451606400",
            "filter": "!-tsS42nh",  # Custom filter to exclude images
            "page":page,
            'pagesize': pagesize,            # Number of questions per request
            'access_token': self.access_token,
            'key': self.app_key
        }

        # Send a GET request to the API
        response = requests.get(url, params=params)
        questions_list = []

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()

            # Create a list to store questions and their details
            questions_answer_list = []

            # Loop through each question in the response
            for question in data['items']:
                if question['is_answered']:
                    if 'accepted_answer_id' in list(question.keys()):
                        question_data= {}
                        for field in ['title', 'link', 'tags', 'question_id', 'is_answered', 'accepted_answer_id',
                                      'view_count', 'answer_count', 'score', 'last_activity_date', 'creation_date'
                                       ]:

                            question_data[f"q_{field}"] = f"{question[field]}"
                        question_data.update(self.__get_accepted_answer(tag, site, question['accepted_answer_id']))
                        questions_answer_list.append(question_data)


            # Convert the list of questions into a Pandas DataFrame for easy manipulation
            #self.qna_df = pd.DataFrame(questions_answer_list)

            return questions_answer_list
        else:
            print(f"Failed to fetch data: {response.status_code}")
            
    # Function to get the accepted answer for a specific question
    def __get_accepted_answer(self, tag, site, answer_id):
        url = f"https://api.stackexchange.com/2.3/answers/{answer_id}"
        params = {
            #"order": "desc",
            #"sort": "votes",
            "tagged": tag,
            "site": site , # Cross Validated (stats.stackexchange.com)
            "filter": "!9_bDE(fI5", #"!-tsS42nh" # Custom filter to include answer body and is_accepted field
            'access_token': self.access_token,
            'key': self.app_key
        }

        # Make the request to the Stack Exchange API
        response = requests.get(url, params=params)
        #return response

        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            # Look for the accepted answer in the response
            answer_data = {}
            for answer in data['items']:
                for field in ['score',  'creation_date', 'body']:
                    if field == 'body':
                        answer_data[f"a_answer"] = self.__get_text_hyperlink(answer[field])
                    else:
                        answer_data[f"a_{field}"] = answer[field]


                return answer_data
            return None  # No accepted answer found
        else:
            print(f"Failed to fetch data: {response.status_code}")
            return None
    
    def __get_text_hyperlink(self, html_body):
    
        # Parse the HTML content
        soup = BeautifulSoup(html_body, 'html.parser')

        # Iterate over all anchor tags and replace them with "text (link)"
        for a in soup.find_all('a', href=True):
            a.replace_with(f"{a.get_text()} ({a['href']})")

        for img in soup.find_all('img'):
            img.decompose() 
         # Remove <a> tags that contain <img> tags
        for link in soup.find_all('a'):
            if link.find('img'):
                link.decompose()  # Removes the entire <a> tag if it contains an <img> tag


        # Get the resulting text including hyperlinks
        clean_text = soup.get_text()

        return clean_text
    
    def fetch_specified_results(self, tag, site, desired_count):
        total_count = self.__get_total_question(tag, site)
        if desired_count > total_count:
            print(f"Warning: Desired count ({desired_count}) exceeds available questions ({total_count}). Adjusting to {total_count}.")
            return None
        results = []
        page = 1
        pagesize = 10
        pbar = tqdm(total=desired_count,desc="Scrapping Data",initial=1)
        while len(results) < desired_count:
            # Fetch Q&A pairs from the current page
            qna_pairs = self.__get_question_answer(tag, site, page, pagesize)

            # Add the fetched results to the list
            results.extend(qna_pairs)

            
            pbar.update( len(qna_pairs))
            page += 1  # Move to the next page
            

            
        #pbar.update(desired_count - pbar.n)
        pbar.set_postfix({"status": "Completed"})
        pbar.close()
        
        # Return exactly the desired number of results
        self.qna_df = pd.DataFrame(results[:desired_count])
        #return self.qna_df 
        
    
    

    

In [8]:
#generated access_token from oauth
access_token = "token"

#stackapps
app_key = 'app_key'

### Create class instance

In [9]:
qna = StackExchangeScrapper(access_token,app_key)

### Scrape data

In [ ]:
qna.fetch_specified_results('neural_networks', 'stats',500)

Scrapping Data:   0%|          | 1/500 [00:00<?, ?it/s]

### Let's check the duplication

In [15]:
qna.qna_df['q_title'].nunique()

458

#### There are 42 duplicate rows,let's filter it.

In [37]:
qna_df = qna.qna_df.drop_duplicates(subset=['q_title'])
qna_df = qna_df.reset_index(drop=True)

### Pre-processing scrapped data using google gemini llm

In [18]:
pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [19]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("googleai")

In [20]:
import google.generativeai as genai
import os

genai.configure(api_key=secret_value_0)

### Build prompt to generate summary

In [21]:
def build_prompt(query):
    txt = """Read the paragraph below and provide a clear, concise summary. The summary should capture the main ideas in fewer than 350 words.

{info}

"""
                      
    
        
    txt = txt.format(info=query)
    #prompt = tokenizer.apply_chat_template(prompt_template, add_generation_prompt=True, return_dict=True, return_tensors="pt").to(model.device)

    return txt


### Define model

In [23]:
model = genai.GenerativeModel("models/gemini-1.0-pro")

### Sample summary

In [38]:
qna_df['q_title'][20]

'What is global max pooling layer and what is its advantage over maxpooling layer?'

In [39]:
qna_df['a_answer'][20]

'Global max pooling  =  ordinary max pooling layer with pool size equals to the size of the input (minus filter size + 1, to be precise). You can see that MaxPooling1D takes a pool_length argument, whereas GlobalMaxPooling1D does not.\nFor example, if the input of the max pooling layer  is $0,1,2,2,5,1,2$, global max pooling outputs $5$, whereas  ordinary max pooling layer with pool size equals to 3 outputs $2,2,5,5,5$ (assuming stride=1).\nThis can be seen in the code (https://github.com/fchollet/keras/blob/3d176e926f848c5aacd036d6095ab015a2f8cc83/keras/layers/pooling.py#L433):\nclass GlobalMaxPooling1D(_GlobalPooling1D):\n    """Global max pooling operation for temporal data.\n    # Input shape\n        3D tensor with shape: `(samples, steps, features)`.\n    # Output shape\n        2D tensor with shape: `(samples, features)`.\n    """\n\n    def call(self, x, mask=None):\n        return K.max(x, axis=1)\n\nIn some domains, such as natural language processing, it is common to use glo

In [40]:
print(model.generate_content(build_prompt(qna_df['a_answer'][20])).text)

**Summary:**

Global max pooling is a variant of the max pooling operation where the pool size equals the input size. In contrast to ordinary max pooling, which outputs a reduced feature map, global max pooling produces a vector that captures the maximum value from the entire input.

The implementation of global max pooling in Keras' `GlobalMaxPooling1D` layer uses the `K.max()` function to compute the maximum value across the time dimension of the input. This results in a vector of maximum values that represents the most significant features in the input sequence.

Global max pooling is frequently employed in natural language processing tasks, where it captures the most salient words or phrases in a text sequence. In contrast, computer vision applications often utilize non-global max pooling to extract localized features from images.


### Get summary for answers

In [ ]:
llm_answer_summary = []
for idx, answer in tqdm(enumerate(qna_df['a_answer'])):
    response = model.generate_content(build_prompt(answer))
    llm_answer_summary.append(response.text)
    # Pause for 60 seconds after every 15 iterations
    if (idx + 1) % 15 == 0:
        print("Pausing for 60 seconds...")
        time.sleep(60)


0it [00:00, ?it/s]

Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...
Pausing for 60 seconds...


In [41]:
qna_df['llm_answer_summary'] = llm_answer_summary

### Actual answer

In [44]:
qna_df.tail(1)['a_answer'][457]

'No, Bayesian network and deep belief network are not the same thing.\n\nBayesian network is a type of probabilistic graphical model (https://en.wikipedia.org/wiki/Graphical_model) where vertexes are random variables and edges are conditional dependencies. For large number of random variables, we use the graphical structure assumptions to decompose the joint distribution in a manageable level. In Bayesian network, there are two major tasks, learning and inference. The ultimate goal of learning is getting the joint distribution of the data, and the goal of inferences is trying to calculate the probability for a given events, assuming you already have the model\nAs mentioned in the link you provided:\n\n\n"Neural networks" is a term usually used to refer to feedforward neural networks. Deep Neural Networks are feedforward Neural Networks with many layers.\n\nYou can think about neural network is just a big non-linear function, that it can approximate complicated outcomes. Here the nodes 

### LLM summary

In [48]:
qna_df.tail(1)['llm_answer_summary'][457]

'Bayesian networks and deep belief networks are distinct types of models:\n\n* **Bayesian networks** are probabilistic graphical models that represent relationships between variables using nodes and edges. They enable efficient handling of large datasets by decomposing joint distributions using graphical assumptions. Their primary tasks are learning (estimating joint distributions) and inference (calculating probabilities).\n\n* **Deep belief networks** are a type of neural network. Neural networks are non-linear functions that model complex outcomes. Their nodes are "neurons" and edges are "connections" that build function components. Deep belief networks have multiple layers, enhancing their approximation capabilities.\n\nIn summary, Bayesian networks focus on probabilistic relationships and efficient computation, while neural networks emphasize non-linear modeling and complex function approximation.'

In [49]:
qna_df.columns

Index(['q_title', 'q_link', 'q_tags', 'q_question_id', 'q_is_answered',
       'q_accepted_answer_id', 'q_view_count', 'q_answer_count', 'q_score',
       'q_last_activity_date', 'q_creation_date', 'a_score', 'a_creation_date',
       'a_answer', 'llm_answer_summary'],
      dtype='object')

### combine tags with pipe operator

In [51]:
ast.literal_eval(qna_df['q_tags'][0])

['model-selection', 'neural-networks']

In [52]:
qna_df['q_tags'] = qna_df['q_tags'].apply(lambda x: "|".join(i.strip() for i in ast.literal_eval(x)))

### Columns rename

In [53]:
qna_df = qna_df.rename(columns={'q_title':'question', 'q_tags':'tags', 'llm_answer_summary':'answer'})

In [54]:
qna_df.head(1)

,question,q_link,tags,q_question_id,q_is_answered,q_accepted_answer_id,q_view_count,q_answer_count,q_score,q_last_activity_date,q_creation_date,a_score,a_creation_date,a_answer,answer
0,How to choose the number of hidden layers and ...,https://stats.stackexchange.com/questions/181/...,model-selection|neural-networks,181,True,1097,1145730,10,820,1661947755,1279584902,671,1280715630,"I realize this question has been answered, but...",**Network Configuration in Neural Networks**\n...


In [55]:
qna.qna_df.to_csv("Stackoverflow_data(neural_networks_stats)_pre_processed_Gemini_LLM.csv",index=False)